## 1 Install conda on Google Colab Pro
- On Colab Pro, select: Runtime -> Change runtime type -> Hardware accelerator --> GPU
- GPU P100 is provided by Google Colab Pro

In [1]:
!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local

--2021-12-24 09:21:24--  https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/archive/Anaconda3-5.1.0-Linux-x86_64.sh [following]
--2021-12-24 09:21:24--  https://repo.anaconda.com/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 577996269 (551M) [application/x-sh]
Saving to: ‘Anaconda3-5.1.0-Linux-x86_64.sh’

Anaconda3-5.1.0-Lin 100%[===================>] 551.22M   185MB/s    in 3.0s    

2021-12-24 09:21:27 (185 MB/s) - ‘Anaconda3-5.1.0-Linux-x86_64.sh’ sa

## 2 Create operating environment by conda

In [2]:
!conda create -n alpha python=3.6 numpy=1.18 pandas=1.0 numba scikit-learn --yes 
!source activate alpha 
!conda install -c bioconda pyteomics python=3.6 --yes
!conda install tensorflow-gpu=1.11 keras=2.2.4 pytorch=1.1.0 cudatoolkit=9.2 -c pytorch --yes

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /usr/local/envs/alpha

  added / updated specs: 
    - numba
    - numpy=1.18
    - pandas=1.0
    - python=3.6
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    readline-8.1 

## 3 Load test data from your Google Drive:
- A folder named Alpha-Tri from [https://github.com/YuAirLab/Alpha-Tri](https://github.com/YuAirLab/Alpha-Tri)
- A folder named workspace containing two files from [https://figshare.com/projects/Alpha-Tri/128000](https://figshare.com/projects/Alpha-Tri/128000)
- Upload these two folders to your Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## 4 Compile the modified DIA-NN

In [4]:
%cd /content/gdrive/MyDrive/Alpha-Tri/DIA-NN_v1.7.12/mstoolkit
!make
!mv diann-alpha /content/gdrive/MyDrive/workspace

/content/gdrive/MyDrive/Alpha-Tri/DIA-NN_v1.7.12/mstoolkit
ar rcs libmstoolkitlite.a adler32.o compress.o crc32.o deflate.o inffast.o inflate.o infback.o inftrees.o trees.o uncompr.o zutil.o xmlparse.o xmlrole.o xmltok.o mzp.MSNumpress.o mzp.mzp_base64_lite.o mzp.BasicSpectrum_lite.o mzp.mzParser_lite.o mzp.RAMPface_lite.o mzp.saxhandler_lite.o mzp.saxmzmlhandler_lite.o mzp.saxmzxmlhandler_lite.o mzp.Czran_lite.o mzp.mz5handler_lite.o mzp.mzpMz5Config_lite.o mzp.mzpMz5Structs_lite.o mzp.BasicChromatogram_lite.o mzp.PWIZface_lite.o Spectrum.o MSObject.o mzMLWriter.o pepXMLWriter.o MSReaderLite.o
g++ -o diann-alpha ../src/diann.cpp -static -Wl,--whole-archive -lpthread -Wl,--no-whole-archive -std=c++17 -lstdc++fs -fpermissive -O3 -mfpmath=sse -msse2 -march=core2 -w -L. -lmstoolkitlite -I. -I./include -D_LARGEFILE_SOURCE -D_FILE_OFFSET_BITS=64 -DGCC -DHAVE_EXPAT_CONFIG_H


## 5 Run DIA-NN

In [9]:
%cd /content/gdrive/MyDrive/workspace
!chmod 777 diann-alpha

/content/gdrive/MyDrive/workspace


In [11]:
%%shell
./diann-alpha --f *.mzML --lib lib.tsv --out diann_out.tsv --threads 4 --qvalue 0.01

DIA-NN 1.8.0 (Data Independent Acquisition by Neural Networks)
Compiled on Dec 24 2021 09:34:12
Current date and time: Fri Dec 24 09:45:45 2021
Logical CPU cores: 2
Thread number set to 4
Output will be filtered at 0.01 FDR

1 files will be processed
[0:00] Loading spectral library lib.tsv
[0:02] Finding proteotypic peptides (assuming that the list of UniProt ids provided for each peptide is complete)
[0:03] Spectral library loaded: 13834 protein isoforms, 13834 protein groups and 192937 precursors in 149971 elution groups.
[0:03] Initialising library
[0:03] Saving the library to lib.tsv.speclib

[0:04] File #1/1
[0:04] Loading run 1h.mzML
[0:21] Detected MS/MS range: 199.998 - 5064.55
[0:21] Run loaded
[0:21] 192937 library precursors are potentially detectable
[0:21] Processing batch #1 out of 96 
[0:21] Precursor search
[0:37] Optimising weights
Averages: 
0.148646 0.143871 0 0 0 0.0625296 0.0643927 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

## 6 Run Prosit to predict the intensities of fragments

In [12]:
%cd /content/gdrive/MyDrive/Alpha-Tri/Prosit
!python prosit.py -lib /content/gdrive/MyDrive/workspace/lib.tsv

/content/gdrive/MyDrive/Alpha-Tri/Prosit
Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/p

## 7 Run Alpha-Tri

In [13]:
%cd /content/gdrive/MyDrive/Alpha-Tri/Alpha-Tri
!python main.py -ws /content/gdrive/MyDrive/workspace --xic --tri

/content/gdrive/MyDrive/Alpha-Tri/Alpha-Tri

Bad key text.latex.unicode in file /usr/local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /usr/local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key nbagg.transparent in file /usr/local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 426 ('nbagg.transparent: True')
You probably need to get an updated matplotlibrc file 